## Installing Haystack


In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Initializing the DocumentStore


In [2]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

## Preparing Documents

Under Summary folder, put the .txt context files.

In [3]:
from haystack.utils import fetch_archive_from_http

doc_dir = "/content/summary"

In [4]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Updating BM25 representation...: 100%|██████████| 4395/4395 [00:00<00:00, 11007.95 docs/s]


{'documents': [<Document: {'content': 'Season-1\nIn this first season, we see Rachel, who has just left her fiancé at the altar, moving in with Monica, her best friend who went to high school with her, and all of the group members who are in college. Chandler is still friends with Ross and welcomes him. The season focuses primarily on the developing relationship between all the characters, with Ross’s unrequited love for Rachel and his commitment to Chandler simultaneously.\n\nSeason-2\nIn the second season, it is seen that the romantic relationship between Rachel and Ross breaks down after a lot of difficulties, there are many obstacles in their relationship, meanwhile, Ross makes a new girlfriend named Julie, and Julie becomes Ross’s girlfriend now. Meanwhile, Rachel becomes jealous of Julie, while Monica begins dating Richard, Richard turns out to be a rich man, and Joey also becomes a soap opera star.\n\nSeason-3\nThe season begins with Ross’s infatuation with Rachel, he can’t help

## Initializing the Retriever


In [6]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

## Initializing the Reader


In [7]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

## Creating the Retriever-Reader Pipeline


In [8]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [9]:
import json

best_returns = []
with open('/content/friends_qa.json', 'r', encoding='utf-8') as file:
    questions_and_answers = json.load(file)


for item in questions_and_answers:
    print(f"Question: {item['Question']}\n Real Answer: {item['Answer']}\n")

    prediction = pipe.run(
    query=item['Question'], params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 5}}
    )

    prediction = [x.to_dict() for x in prediction["answers"]]

    best_answer = prediction[0]['answer']
    best_answer_score = prediction[0]['score']
    best_answer_context = prediction[0]['context']

    best_return = {
        "Question": item['Question'],
        "Answer": item['Answer'],
        "Best_Answer": best_answer,
        "Best_Answer_Score": best_answer_score,
        "Best_Answer_Context": best_answer_context
    }
    best_returns.append(best_return)

Question: Joey played Dr. Drake Ramoray on which soap opera show?
 Real Answer: Days of Our Live.



Inferencing Samples: 100%|██████████| 1/1 [00:04<00:00,  4.66s/ Batches]


Question: How many times did Ross get divorced?
 Real Answer: Three.



Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.40s/ Batches]


Question: What store does Phoebe hate?
 Real Answer: Pottery Barn.



...

Inferencing Samples: 100%|██████████| 1/1 [00:05<00:00,  5.65s/ Batches]


Question: What did Rachel get a tattoo of?
 Real Answer: A heart.



Inferencing Samples: 100%|██████████| 1/1 [00:05<00:00,  5.44s/ Batches]


Question: Who is the first Friend to speak on the show?
 Real Answer: Monica.



Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.28s/ Batches]


In [11]:
with open('/content/best_returns.json', 'w', encoding='utf-8') as file:
    json.dump(best_returns, file, ensure_ascii=False, indent=4)